In [5]:
import os
ROOT_PATH = '/sources/dataset'
VER = 'ver1'
SAVE_PATH = os.path.join(ROOT_PATH, 'features', VER)
os.makedirs(SAVE_PATH, exist_ok=True)

description = \
    """ ver 1. \n 
        얼굴 랜드마크:
        LIP = [
            61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
            291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
            95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
        ]

        포즈 랜드마크: 
        사용 안함

        손 랜드마크: 전부 사용

        최종 데이터 형태: [max_len, 82, 3]

        데이터 생성 파이프라인 요약
        1. 전체 시간프레임에 대한 mean과 std로 normalization
        2. nan -> 0
        3. max length 만큼만 return
    """

with open(os.path.join(ROOT_PATH, 'features', VER, 'description.txt'), 'w') as f:
    f.write(description)

In [6]:
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.ndimage import zoom
from tqdm import tqdm

In [4]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

### Feature Generation class

In [17]:
max_length = 80
num_lm = 82

class InputNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.max_length = max_length 
  
    def forward(self, xyz):
        xyz = xyz - xyz[~torch.isnan(xyz)].mean(0,keepdim=True) #noramlisation to common maen
        xyz = xyz / xyz[~torch.isnan(xyz)].std(0, keepdim=True)

        LIP = [
            61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
            291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
            95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
        ]

        lip = xyz[:, LIP]
        lhand = xyz[:, 468:489]
        rhand = xyz[:, 522:543]
        xyz = torch.cat([  # (none, 82, 3)
            lip,
            lhand,
            rhand,
        ], 1)
        xyz[torch.isnan(xyz)] = 0
        x = xyz[:self.max_length]
        return x

feat_converter = InputNet()

### Convert raw data and save

In [28]:
from tqdm import tqdm

def convert_row(row, right_handed=True):
    x = load_relevant_data_subset(os.path.join(ROOT_PATH, row.path))
    x = torch.tensor(x, dtype=torch.float32)
    x = feat_converter(x).cpu().numpy()
    return x, row.label

def convert_and_save_data():
    df = pd.read_csv(os.path.join(ROOT_PATH, 'train.csv'))
    label_map = json.load(open(os.path.join(ROOT_PATH, 'sign_to_prediction_index_map.json')))
    df['label'] = df['sign'].map(label_map)

    total = df.shape[0]

    npdata = np.zeros((total, max_length, num_lm, 3))
    nplabels = np.zeros(total)
    nppids = np.zeros(total)

    for i, row in tqdm(enumerate(df.itertuples()), total=total):
        (x, y) = convert_row(row)
        npdata[i, :x.shape[0]] = x
        nplabels[i] = y
        nppids[i] = int(row.participant_id)

        if i == 1:
            break
        # if i == total - 1:
        #     break
    
    # np.save(os.path.join(SAVE_PATH, 'feature_data.npy'), npdata)
    # np.save(os.path.join(SAVE_PATH, 'feature_labels.npy'), nplabels)
    # np.save(os.path.join(SAVE_PATH, 'participants.npy'), nppids)

In [29]:
convert_and_save_data()

  0%|          | 1/94477 [00:00<35:32, 44.30it/s]


In [30]:
X = np.load(os.path.join(SAVE_PATH, 'feature_data.npy'))
y = np.load(os.path.join(SAVE_PATH, 'feature_labels.npy'))
print(X.shape, y.shape)

print(X[0, :].shape, X[0, :])

(94477, 80, 82, 3) (94477,)
(80, 82, 3) [[[ 0.53944099  0.29848355 -0.87295991]
  [ 0.53821802  0.28912947 -0.90358984]
  [ 0.54292607  0.27921778 -0.93584406]
  ...
  [ 0.05829294  0.36396381 -1.0776062 ]
  [ 0.17296308  0.32440293 -1.10091221]
  [ 0.28166935  0.32911116 -1.1148628 ]]

 [[ 0.5305692   0.3058185  -0.87093896]
  [ 0.53642601  0.29671487 -0.90204638]
  [ 0.54879284  0.28625819 -0.93556219]
  ...
  [ 0.12695304  0.29156283 -1.11008441]
  [ 0.2557334   0.26448274 -1.13670397]
  [ 0.37650952  0.27268359 -1.15147626]]

 [[ 0.51968706  0.31230304 -0.86367387]
  [ 0.52490693  0.30063587 -0.89527011]
  [ 0.53764594  0.28717068 -0.93001246]
  ...
  [ 0.19723418  0.27750027 -1.10325122]
  [ 0.32734609  0.25656015 -1.13265097]
  [ 0.44683465  0.26329887 -1.15792537]]

 ...

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 